In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, month
import json

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Dynamic Data Analysis") \
    .getOrCreate()

# Read configuration file
with open("config.json", "r") as f:
    config = json.load(f)

# Read datasets specified in the configuration
datasets = {}
for dataset_name, dataset_info in config["datasets"].items():
    dataset = spark.read.format(dataset_info["format"]).load(dataset_info["path"])
    datasets[dataset_name] = dataset.selectExpr(*[f"{col_name} as {new_col_name}" for col_name, new_col_name in dataset_info["schema"].items()])

# Perform analyses specified in the configuration
for analysis in config["analyses"]:
    dataset = datasets[analysis["dataset"]]
    result_df = dataset.groupBy(analysis["group_by"]).agg(*[getattr(dataset, agg["function"])(col(agg["column"])).alias(agg["column"]) for agg in analysis["aggregations"]])
    if "order_by" in analysis:
        result_df = result_df.orderBy(col(analysis["order_by"]["column"]), ascending=(analysis["order_by"]["order"].lower() == "asc"))
    if "limit" in analysis:
        result_df = result_df.limit(analysis["limit"])
    result_df.show()

# Stop SparkSession
spark.stop()


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `OrderID` cannot be resolved. Did you mean one of the following? [`_c0`, `_c1`, `_c2`].; line 1 pos 0;
'Project ['OrderID AS integer#23, 'CustomerID AS string#24, 'OrderDate AS date#25]
+- Relation [_c0#17,_c1#18,_c2#19] csv
